In [1]:
%pip install -U trl numpy peft transformers datasets bitsandbytes huggingface_hub

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.0/138.0 kB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 245.2/245.2 kB 11.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 251.6/251.6 kB 16.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.0/9.0 MB 94.9 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 28.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.8/119.8 MB 13.8 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 401.2/401.2 kB 26.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 79.2 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 102.4/102.4 kB 6.8 MB/s eta 0:00:00
  Attempting uninstall: huggingface_hub
    Found existing installation: huggingface-hub 0.22.2
    Uninstalling huggingface-hub-0.22.2:
      Successfully uninstalled huggingface-hub-0.22.2
  Attempting uninstall: tokenizers
    Found existing installa

In [2]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import torch
from datasets import load_dataset, Dataset
from scipy.special import softmax
from peft import prepare_model_for_kbit_training, LoraConfig, get_peft_model
from sklearn.metrics import accuracy_score, f1_score, log_loss, confusion_matrix
from transformers import set_seed, TrainingArguments, Trainer, AutoTokenizer, AutoModelForSequenceClassification, BitsAndBytesConfig, DataCollatorWithPadding
from huggingface_hub import notebook_login, login
import json
import pandas as pd
import os

2024-05-17 14:12:07.887464: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-05-17 14:12:07.887560: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-05-17 14:12:08.065894: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


cuda
Number of GPUs available: 2


In [5]:
login()
#notebook_login()

In [6]:
#quantization configurations - so you quantize the model while inferencing
bnb_config = BitsAndBytesConfig(
    load_in_4bit = True,
    bnb_4bit_qunat_type = "nf4",
    bnb_4bit_compute_dtype = torch.float16,
)

'''Use model_name = 'meta-llama/Llama-2-7b' for llama-2 and 
model_name = 'meta-llama/Meta-Llama-3-8B' for llama-3 if needed, 
make sure you are updated with lmsys leaderboard to use the best models 
at the time you are using this notebook'''

model_name = 'meta-llama/Meta-Llama-3-8B'

tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.pad_token = tokenizer.bos_token

model = AutoModelForSequenceClassification.from_pretrained(
    model_name,
    num_labels=4, # MMLU is a multiple-choice test with 4 answers
    quantization_config=bnb_config, 
    device_map="auto",
    #trust_remote_code=True,
    #token=token
    )
model.config.pad_token_id = tokenizer.pad_token_id
#model.config.use_cache = False


# Initial trainable parameters of our model.
def count_trainable_params(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)


num_params = count_trainable_params(model)
formatted_num_params = "{:,}".format(num_params)
print(f"Number of trainable parameters: {formatted_num_params}")

Unused kwargs: ['bnb_4bit_qunat_type']. These kwargs are not used in <class 'transformers.utils.quantization_config.BitsAndBytesConfig'>.
/opt/conda/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/50.6k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/73.0 [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


...

model-00003-of-00004.safetensors:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

model-00004-of-00004.safetensors:   0%|          | 0.00/1.17G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

Some weights of LlamaForSequenceClassification were not initialized from the model checkpoint at meta-llama/Meta-Llama-3-8B and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Using 2 GPUs
Number of trainable parameters: 525,619,200


In [ ]:
# Write a Text Generation Function
#def generate_text(prompt, max_length=150):
#    inputs = tokenizer(prompt, return_tensors="pt")
#    
#    #output = model(**inputs)
#    
#    with torch.no_grad():
#        outputs = model.generate(inputs.input_ids, max_length=max_length)
#    return tokenizer.decode(outputs[0], skip_special_tokens=True)

In [ ]:
#prompts = ["Hello, how are you?", "Explain the Massive Multitask Language Understanding (MMLU) Benchmark."]

#for idx, prompt in enumerate(prompts):
#    output = generate_text(prompt)
#    print(f"Prompt No. {idx+1}: '{prompt}'\n - Answer: '{output}'\n\n")

In [9]:
# Load the Fine Tuning Dataset
def load_jsonl(file_path):
    with open(file_path, 'r') as file:
        data = [json.loads(line) for line in file]
    return data

In [10]:
# Function to preprocess the data
def preprocess_data(data):
    preprocessed_data = []
    for entry in data:
        question = entry['question']['stem']
        choices = entry['question']['choices']
        answer_key = entry['answerKey']
        
        # Format the question and choices into a single input string
        formatted_question = question + "\n"
        for choice in choices:
            formatted_question += f"{choice['label']}: {choice['text']}\n"
        
        # Convert the answer key from letter to numerical index
        correct_answer_idx = ord(answer_key) - ord('A')  # Convert 'A', 'B', 'C', 'D' to 0, 1, 2, 3

        # Check if the correct_answer_idx is within the valid range
        if 0 <= correct_answer_idx <= 3:
            preprocessed_data.append({
                'input': formatted_question.strip(),
                'label': correct_answer_idx,  # Numerical label for training
                'original_label': answer_key  # Original label for display
            })
        else:
            #print(f"Skipping invalid entry: {question}\n With Choises: {choices}\n And answer key {answer_key} resulting in index {correct_answer_idx}\n\n")
            print(f"Skipping invalid entry: Answer key {answer_key} resulting in index {correct_answer_idx}")

        
    return preprocessed_data

In [11]:
# Load and preprocess the data
train_data = load_jsonl('/kaggle/input/ai2-arc/ARC-Challenge/ARC-Challenge-Train.jsonl')
dev_data = load_jsonl('/kaggle/input/ai2-arc/ARC-Challenge/ARC-Challenge-Dev.jsonl')
test_data = load_jsonl('/kaggle/input/ai2-arc/ARC-Challenge/ARC-Challenge-Test.jsonl')

train_data = preprocess_data(train_data)
dev_data = preprocess_data(dev_data)
test_data = preprocess_data(test_data)

Skipping invalid entry: A chemical property of a mineral is evident if the mineral
 With Choises: [{'text': 'breaks easily when struck with a hammer', 'label': '1'}, {'text': 'bubbles when acid is placed on it', 'label': '2'}, {'text': 'is easily scratched by a fingernail', 'label': '3'}, {'text': 'reflects light from its surface', 'label': '4'}]
 And answer key 2 resulting in index -15


Skipping invalid entry: Several tomato plants are grown indoors next to a sunny window. The plants receive water and fertilizer and remain on the windowsill. What will most likely happen?
 With Choises: [{'text': 'Most of the leaves on the window side will wilt and die.', 'label': '1'}, {'text': 'The roots of the plants will grow upward from the soil.', 'label': '2'}, {'text': 'Water droplets will collect on the leaves facing away from the window.', 'label': '3'}, {'text': 'The stem will bend toward the window.', 'label': '4'}]
 And answer key 4 resulting in index -13


Skipping invalid entry: Weather

In [12]:
# Convert to DataFrame for easy manipulation and export
train_df = pd.DataFrame(train_data)
dev_df = pd.DataFrame(dev_data)
test_df = pd.DataFrame(test_data)

In [13]:
train_df.head(10)

,input,label,original_label
0,George wants to warm his hands quickly by rubb...,0,A
1,Which of the following statements best explain...,1,B
2,A fold observed in layers of sedimentary rock ...,1,B
3,Which of these do scientists offer as the most...,3,D
4,A boat is acted on by a river current flowing ...,1,B
5,Which land form is the result of the construct...,1,B
6,Which statement best compares single-celled an...,2,C
7,"As part of an experiment, an astronaut takes a...",3,D
8,Which of the following is a trait that a dog d...,2,C
9,Hatchling sea turtles are typically dark in co...,2,C


In [14]:
# Convert preprocessed data to Hugging Face Dataset
train_dataset = Dataset.from_pandas(train_df)
dev_dataset = Dataset.from_pandas(dev_df)
test_dataset = Dataset.from_pandas(test_df)

In [15]:
# Tokenize the input
def tokenize_function(examples):
    tokenized_inputs = tokenizer(examples['input'], padding='max_length', truncation=True)
    tokenized_inputs['original_label'] = examples['original_label']
    return tokenized_inputs

In [16]:
train_dataset = train_dataset.map(tokenize_function, batched=True)
dev_dataset = dev_dataset.map(tokenize_function, batched=True)

test_dataset = test_dataset.map(tokenize_function, batched=True)


train_dataset.set_format(type='torch', columns=['input_ids', 'attention_mask', 'label', 'input', 'original_label'])
dev_dataset.set_format(type='torch', columns=['input_ids', 'attention_mask', 'label', 'input', 'original_label'])
test_dataset.set_format(type='torch', columns=['input_ids', 'attention_mask', 'label', 'input', 'original_label'])

Map:   0%|          | 0/1096 [00:00<?, ? examples/s]

Asking to pad to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no padding.
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


Map:   0%|          | 0/294 [00:00<?, ? examples/s]

Map:   0%|          | 0/1150 [00:00<?, ? examples/s]

In [21]:
# Function to classify and print example questions
def classify_and_print_examples(model, dataset, tokenizer, num_examples=10):
    # Select a subset of examples
    examples = dataset.select(range(num_examples))

    for i, example in enumerate(examples):
        # Tokenize the input text
        inputs = tokenizer(example['input'], return_tensors="pt", padding=True, truncation=True)

        outputs = model(**inputs)
        
        logits = outputs.logits.to(torch.float32)  # Cast logits to float32
        probabilities = torch.softmax(logits, dim=-1)
        predicted_class = torch.argmax(probabilities, dim=-1).item()
        confidence = probabilities[0][predicted_class].item()
        
        print(f"Example {i + 1}:\n{example['input']}\n")
        print(f"Predicted Class: {chr(65 + predicted_class)} with confidence {confidence:.2f}\n")
        print(f"Correct Answer: {example['original_label']}\n")
        print("-" * 80 + "\n")"

In [22]:
# Ensure the model is on the correct device
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model.to(device)

# Show examples from the test set
classify_and_print_examples(model, test_dataset, tokenizer)

RuntimeError: Caught RuntimeError in replica 0 on device 0.
Original Traceback (most recent call last):
  File "/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/parallel_apply.py", line 85, in _worker
    output = module(*input, **kwargs)
  File "/opt/conda/lib/python3.10/site-packages/torch/nn/modules/module.py", line 1518, in _wrapped_call_impl
    return self._call_impl(*args, **kwargs)
  File "/opt/conda/lib/python3.10/site-packages/torch/nn/modules/module.py", line 1527, in _call_impl
    return forward_call(*args, **kwargs)
  File "/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/data_parallel.py", line 185, in forward
    outputs = self.parallel_apply(replicas, inputs, module_kwargs)
  File "/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/data_parallel.py", line 200, in parallel_apply
    return parallel_apply(replicas, inputs, kwargs, self.device_ids[:len(replicas)])
  File "/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/parallel_apply.py"

In [ ]:
# Define the compute metrics function
def compute_metrics(p):
    predictions, labels = p.predictions, p.label_ids
    predictions = np.argmax(predictions, axis=1)
    accuracy = (predictions == labels).mean()
    return {"accuracy": accuracy}

In [ ]:
# Load the MMLU dataset
mmlu_dataset = load_dataset('lukaemon/mmlu', 'high_school_european_history')
mmlu_dataset

In [ ]:
mmlu_dataset['test'][0]

In [ ]:
# Preprocess the MMLU dataset
def preprocess_mmlu(data):
    preprocessed_data = []
    for entry in data:
        question = entry['input']
        choices = [entry['A'], entry['B'], entry['C'], entry['D']]
        answer_key = entry['target']

        formatted_question = question + "\n"
        for i, choice in enumerate(choices):
            formatted_question += f"{chr(65 + i)}: {choice}\n"

        correct_answer_idx = ord(answer_key) - ord('A')  # Convert 'A', 'B', 'C', 'D' to 0, 1, 2, 3

        preprocessed_data.append({
            'input': formatted_question.strip(),
            #'label': answer_key
            'label': correct_answer_idx,  # Numerical label for training
            'original_label': answer_key  # Original label for display
        })
    return preprocessed_data

In [ ]:
preprocessed_mmlu = preprocess_mmlu(mmlu_dataset['test'])  # Use the 'test' split for evaluation
mmlu_df = pd.DataFrame(preprocessed_mmlu)
mmlu_dataset = Dataset.from_pandas(mmlu_df)

In [ ]:
# Apply the tokenization function
mmlu_dataset = mmlu_dataset.map(tokenize_function, batched=True)

# Ensure the original input and label are retained in the dataset format
mmlu_dataset.set_format(type='torch', columns=['input_ids', 'attention_mask', 'label', 'input', 'original_label'])

In [ ]:
mmlu_dataset[0]

In [ ]:
# Create a data collator that dynamically pads the inputs
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [ ]:
'''
==========================================================================================
After extensive trainings and experience my suggestions are,
use only (k, q, v, o) projections or else it may overfitt,
For DoRA as paper suggests lower rank may be optimal my choice is (r=8, alpha=8 or 16)
If you increase rank (r=16) and alpha(16<) try dropout (0.2).
==========================================================================================
Lastly, it all boils down to your specific use case. 
Experiment with your own choices to grasp your own hyperparameters for your task.
==========================================================================================
'''

peft_config = LoraConfig(
        r=16,
        lora_alpha=64,
        lora_dropout=0.2,
        bias="none",
        task_type='SEQ_CLS',
        use_dora=True, # Make this False if you wanna use only LoRA
        target_modules=[
    "up_proj",
    "o_proj",
    "v_proj",
    "gate_proj",
    "q_proj",
    "down_proj",
    "k_proj"
  ]
) 

In [ ]:
# Change the parameters and hyper-parameters as per your use case.
epochs = 5 # Increase for longer training with better results
batch_size = 16 # Higher Batch Size -> Faster Training, but more VRAM required 48 seems to be the sweet spot for 16GB
gradient_accumulation_steps = 4 # Increase to 4 or even more for better results

# output dir 
model_version = "openchat_3.5_QLoRA"
model_dir = f"{model_version}"

training_args = TrainingArguments(
    run_name=model_version,
    logging_dir=f"{model_dir}/logs",
    output_dir=model_dir,
    logging_steps=100, # Increase to 100 for better resuts
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    gradient_accumulation_steps=gradient_accumulation_steps,
    evaluation_strategy="epoch",
    learning_rate=2e-4,# set to 2e-6 for faster fine tuning. Increase to 2e-5 or 2e-4 for better results
    num_train_epochs=epochs,
    lr_scheduler_type="constant",
    save_strategy="epoch",
    fp16=True,
    report_to='none',  # Disable WandB integration
    #use_cache=False  # Disable cache due to gradient checkpointing
)

#model.gradient_checkpointing_enable() # Comment this out to speed up the fine tuning
model = prepare_model_for_kbit_training(model)
model = get_peft_model(model, peft_config)

# The parameters after appling LoRA
num_params = count_trainable_params(model)
formatted_num_params = "{:,}".format(num_params)
print(f"Number of trainable parameters: {formatted_num_params}")

In [ ]:
# Function to evaluate the model
def evaluate_model(model, dataset):
    training_args = TrainingArguments(
        output_dir='./results',
        per_device_eval_batch_size=8,
        logging_dir='./logs',
        do_train=False,
        do_eval=True,
        logging_steps=25,
        report_to='none',  # Disable WandB integration
    )

    trainer_eval = Trainer(
        model=model,
        args=training_args,
        eval_dataset=dataset,
        data_collator=data_collator,
        compute_metrics=compute_metrics,
    )

    return trainer_eval.evaluate()

In [ ]:
print("Evaluating original model on MMLU benchmark...")
original_eval_results = evaluate_model(model, mmlu_dataset)
print(f"Original Model Results: {original_eval_results}")

In [ ]:
print(train_dataset['label'].unique())
print(dev_dataset['label'].unique())

In [ ]:
# Configure Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=dev_dataset,  # Use dev_dataset for evaluation during training
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

# Start training
trainer.train()

# Evaluate the model on the test set
eval_results_ft = trainer.evaluate(test_dataset)
print(f"Test Evaluation Results: {eval_results}")

In [ ]:
# save trained model
trainer.save_model(f'{model_dir}/model')